# Image Segmentation - 2nd Challenge

## UNet Architecture

In this notebook we create an architecture based on UNet.







Main aspects:
- Skip connections between the encoder and the decoder
- Dice loss implementation



In [ ]:
import numpy as np 
import pandas as pd 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os


import tensorflow as tf
import numpy as np


SEED = 1234
tf.random.set_seed(SEED)  


cwd = os.getcwd()


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(featurewise_center=True,
                                            featurewise_std_normalization=True,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            rescale=1./255,
                                            validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(featurewise_center=True,
                                             featurewise_std_normalization=True,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             rescale=1./255,
                                             validation_split=0.2)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
dataset_dir = "/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset"

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256


#------------------- TRAINING DATASET ---------------------

training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       #color_mode = 'rgb',
                                                       subset='training')  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, 
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         seed=SEED,
                                                         color_mode='grayscale',
                                                         subset='training')
train_gen = zip(train_img_gen, train_mask_gen)



#------------------- VALIDATION DATASET ---------------------

valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       #color_mode = 'rgb',
                                                       subset='validation')
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, 
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         seed=SEED,
                                                         color_mode='grayscale',
                                                         subset='validation')
valid_gen = zip(valid_img_gen, valid_mask_gen)

In [ ]:
def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_


#------------------- TRAINING DATASET ---------------------

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))



train_dataset = train_dataset.map(prepare_target)

train_dataset = train_dataset.repeat()



#------------------- VALIDATION DATASET ---------------------

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

valid_dataset = valid_dataset.map(prepare_target)

valid_dataset = valid_dataset.repeat()

In [ ]:
from tensorflow import keras


# Main structure:
#   - Conv2D
#   - BatchNorm
#   - ReLU


# ----- ENCODER
def encoder_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.BatchNormalization()(c)
    c = keras.layers.ReLU()(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.BatchNormalization()(c)
    c = keras.layers.ReLU()(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p



# ----- DECODER
def decoder_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(concat)
    c = keras.layers.BatchNormalization()(c)
    c = keras.layers.ReLU()(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    return c



# ----- BOTTOM
def bottom_layer(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.BatchNormalization()(c)
    c = keras.layers.ReLU()(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    return c

In [ ]:
def UNet():
    
    #Filters
    f = [8, 16, 32, 64, 128, 256]
    
    #Input
    inputs = keras.layers.Input((img_h, img_w, 3))
    
    p0 = inputs
    
    ### --- ENCODER  ###
    c1, p1 = encoder_block(p0, f[0]) 
    c2, p2 = encoder_block(p1, f[1]) 
    c3, p3 = encoder_block(p2, f[2])
    c4, p4 = encoder_block(p3, f[3]) 
    c5, p5 = encoder_block(p4, f[4])
    
    
    bn = bottom_layer(p5, f[5])
    
    
    ### --- DECODER --- ###
    u0 = decoder_block(bn, c5, f[4])
    u1 = decoder_block(u0, c4, f[3]) 
    u2 = decoder_block(u1, c3, f[2]) 
    u3 = decoder_block(u2, c2, f[1]) 
    u4 = decoder_block(u3, c1, f[0]) 
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    
    
    return model

### Implementation of Dice Loss

In [ ]:
from keras import backend as K


def dice_coef(y_true, y_pred, smooth=1):
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

### Implementation of Intersection over Union

In [ ]:
def my_IoU(y_true, y_pred):
    
    y_pred = tf.cast(y_pred > 0.5, tf.float32) 

    intersection = tf.reduce_sum(y_true * y_pred)
    
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    return intersection / union

In [ ]:
from tensorflow.keras.optimizers import *

model = UNet()

model.compile(optimizer = Adam(lr = 1e-3), loss=dice_coef_loss, metrics=[my_IoU])
model.summary()

In [ ]:
model.fit(x=train_dataset,
          epochs=30,
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen)) 

In [ ]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [ ]:
from PIL import Image

test_dir = "/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset/test"
test_img_dir = os.path.join(test_dir, 'images/img')

img_filenames = next(os.walk(test_img_dir))[2]
results = {}

for image_name in img_filenames:
    img = Image.open(os.path.join(test_img_dir,'{}').format(image_name)).convert('RGB')
    img = img.resize((img_h, img_w))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0) 
    
    out = model.predict(x=img_array / 255.)
    
    out = np.round(out)
    
    predicted = rle_encode(out)
    
    name = os.path.splitext(image_name)[0]
    
    results[name] = predicted
 
create_csv(results)